# Clustering Exploratory Data Analysis
Notebook to perform clustering analysis on data on BigQuery.<br>

## Imports

In [ ]:
import json
import calendar
import datetime
import warnings
import numpy as np
import pandas as pd
from datetime import timedelta
from collections import Counter
from google.cloud import bigquery

from tqdm.auto import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

import scipy.stats
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.seasonal import seasonal_decompose

warnings.filterwarnings('ignore')
tqdm.pandas()

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, Attention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm.keras import TqdmCallback
from tensorflow.keras.utils import timeseries_dataset_from_array

## Config

In [ ]:
# Config

PROJECT_ID = 'arpae-prod-ml'

# BigQuery
JOINED_BQ_DATASET = 'JOINED_DATA'
JOINED_DATA_TABLE = "ALL_METEO_FEATS_POL_DAT"
JOINED_WEEK_DATA_TABLE = "ALL_METEO_WEEK_FEATS_POL_DAT"


## Methods

In [ ]:
# # Read Methods

def _run_query(client, query): 
    df = client.query(query).to_dataframe()
    return df

def _create_station_widget(df):
    station_ids = df.station_id.sort_values().unique()
    station_wdgt = widgets.Dropdown(options=station_ids, description='Station Id:', layout={"width":"50%"})
    return station_wdgt

def _create_pol_var_id_widget(df):
    pol_var_ids = df.pol_var_id.sort_values().unique()
    pol_wdgt = widgets.Dropdown(options=pol_var_ids, description='Pol var id:', layout={"width":"50%"})
    return pol_wdgt

def min_max_scale(x):
    return (x - x.min()) / (x.max() - x.min())

# 1. Read Data

## 1.1 Config BigQuery

In [ ]:
# Setup Client

bq_client = bigquery.Client()
bq_client

## 1.2 Read Tables

In [ ]:
# Load B_CODES

sql = f"SELECT * FROM `{PROJECT_ID}.SAMPLE_DATA.B_CODES` WHERE eligible IS true"
b_codes = _run_query(bq_client, sql)

print(b_codes.shape)
b_codes.head(3)

In [ ]:
# Load BCODE_SOGLIE

sql = f"SELECT * FROM `{PROJECT_ID}.SAMPLE_DATA.BCODE_SOGLIE`"
b_codes_soglie = _run_query(bq_client, sql)
b_codes_soglie.head(3)

In [ ]:
# Load all meteo and pollen data

sql = f"""
    SELECT DISTINCT *
    FROM `{PROJECT_ID}.{JOINED_BQ_DATASET}.{JOINED_DATA_TABLE}`
    WHERE pol_var_id IN {tuple(b_codes.var_id)}
    ORDER BY station_id, pol_var_id, date
"""

raw_df = _run_query(bq_client, sql)

print(raw_df.shape)
raw_df.head(3)

## 1.3 Load clustering results

In [ ]:
clusters = pd.read_csv("data/clusters/clustering_tsd_intervals.csv")

print(clusters.shape)
clusters.sample(3)

In [ ]:
plt.figure(figsize=(8,4))
plt.title("Number of specie/station per cluster")
sns.countplot(x=clusters["cluster"]);

# 2. Data preprocess

In [ ]:
raw_df.sort_values(["pol_var_id", "date"], inplace=True)

In [ ]:
# Convert dates to datetime format
raw_df.date = pd.to_datetime(raw_df.date)

raw_df["month"] = raw_df.date.dt.month
# df["day"] = df.date.dt.day

In [ ]:
# Set datetime as index
# raw_df.set_index(["pol_var_id", "station_id", "date"], inplace=True)
raw_df.set_index("date", inplace=True)

raw_df.head(3)

In [ ]:
### Sort dataset
raw_df.sort_values(by=['station_id', "pol_var_id", 'date'], ascending=True, inplace=True)

# 3. Feature process

## 3.1 Select features

In [ ]:
# print(raw_df.keys())

In [ ]:
# print(sorted(raw_df.pol_var_id.unique()))

In [ ]:
# print(sorted(raw_df.station_id.unique()))

In [ ]:
# station_label = "station_id"

features = [
    "station_id",
    "pol_var_id",
    "month",
    "seasonal",
    #"trend",
    "station_lat", "station_lon",
    "station_H_piano_strada", "station_H_mslm",
    "B13011_min", "B13011_max", "B13011_mean", "B13011_std",
    "B14198_min", "B14198_max", "B14198_mean", "B14198_std",
    "TEMP_min", "TEMP_max", "TEMP_mean", "TEMP_std",
    "PREC"
]

y_label = "pol_value"

## 3.2 Adding Seasonal trend as feature

In [ ]:
raw_df["seasonal"] = None
for (stat_id, specie_id), specie_data in tqdm(raw_df.groupby(["station_id", "pol_var_id"])):
    
    # Prepare specie data
    specie_data.sort_index(inplace=True)
    specie_pollen = specie_data[["pol_value"]].resample('D').interpolate()
    
    # Evaluate seasonal
    decomposition = seasonal_decompose(specie_pollen, period=365, model='additive')
    specie_seasonal = decomposition.seasonal
    
    # Join seasonal feautre
    mask = ((raw_df.station_id == stat_id) & (raw_df.pol_var_id == specie_id)) # get specie mask
    specie_seasonal = specie_seasonal.reindex(raw_df.loc[mask].index) # reindex due to index diff
    raw_df.loc[mask, "seasonal"] = specie_seasonal # now we can add the seasonal feature

raw_df.seasonal = raw_df.seasonal.astype(float)

# 4. Select data and generate windows

### 4.1 Select window size

In [ ]:
window_day_size = 30

### 4.2 Select Cluster

In [ ]:
cluster_n = 7

specie_cluster = clusters[clusters.cluster == cluster_n].sample(5) ################################################## DEL ME
specie_cluster

### 4.3 Select data

In [ ]:
# Select species/stations from cluster result
dataset = pd.merge(raw_df, specie_cluster, on=["station_id", "pol_var_id"], how="inner")
dataset = dataset[features + [y_label]]
dataset.station_id = dataset.station_id.astype(int)
dataset.dropna(inplace=True)

assert len(specie_cluster) == len(dataset[["station_id", "pol_var_id"]].drop_duplicates())

print(dataset.shape)
dataset.head(5)

### 4.4 Shift pollen value to predict by one day

In [ ]:
### Applying shift here
dataset['pol_value'] = dataset.groupby(['station_id', 'pol_var_id'])['pol_value'].shift(-1)
dataset.dropna(inplace=True)

### 4.5 Specie is added as a categorical feature to our model

In [ ]:
le = LabelEncoder()
dataset["pol_var_id"] = le.fit_transform(dataset["pol_var_id"])
dataset.pol_var_id.unique()

### 4.6 Training and Test set split

In [ ]:
X = dataset[features]
y = dataset.pol_value

split_idx = int(len(X) * .95)

x_train, y_train = X[:split_idx], y[:split_idx]
x_test, y_test = X[split_idx:], y[split_idx:]

x_train.shape, y_train.shape, x_test.shape, y_test.shape

### 4.7 Generate windows for LSTM, in batches

In [ ]:
batch_size = 64

train_dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
    x_train,
    y_train,
    sequence_length=window_day_size,
    batch_size=batch_size,
)

test_dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
    x_test,
    y_test,
    sequence_length=window_day_size,
    batch_size=batch_size,
)

# 5. Model training

In [ ]:
input_shape = (window_day_size, len(features))

model = tf.keras.Sequential([
    Bidirectional(LSTM(units=512, input_shape=input_shape, return_sequences=True)),
    Dropout(.2),
    Bidirectional(LSTM(units=512, return_sequences=True)),
    Dropout(.2),
    Bidirectional(LSTM(units=512)),
    Dropout(.2),
    Dense(units=64, activation='relu'),
    Dense(1)
])

n_epochs = 10
learning_rate = .001

optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer="adam", loss='mse')

# early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('data/model/best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    #batch_size=batch_size,
    epochs=n_epochs,
    shuffle=False,
    verbose=0,
    # callbacks=[early_stop, model_checkpoint, TqdmCallback(verbose=1)]
    callbacks=[model_checkpoint, TqdmCallback(verbose=1)]
)

# Load the best model
# best_model = tf.keras.models.load_model('data/model/best_model.h5')

In [ ]:
# Plot the MSE history
plt.figure(figsize=(10,3))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend(['loss', 'val_loss'], loc='upper right')
plt.grid(alpha=.4)
plt.show()

In [ ]:
### Test on training data
predictions = model.predict(train_dataset)
predictions = predictions.reshape(predictions.shape[0])
mse = model.evaluate(train_dataset, verbose=False)

plt.figure(figsize=(16,4))
plt.title(f"x_train Preditcions - MSE: {np.round(mse, 2)}")
plt.plot(y_train.values, label="pol_value true")
plt.plot(predictions, label="pol_value predicted")
plt.legend(loc="upper right")
plt.grid(alpha=.4)

In [ ]:
### Test on validation data
predictions = model.predict(test_dataset)
predictions = predictions.reshape(predictions.shape[0])
mse = model.evaluate(test_dataset, verbose=False)

plt.figure(figsize=(16,4))
plt.title(f"x_test Preditcions - MSE: {np.round(mse, 2)}")
plt.plot(y_test.values, label="pol_value true")
plt.plot(predictions, label="pol_value predicted")
plt.legend(loc="upper right")
plt.grid(alpha=.4)